In [ ]:
!pip install transformers
import numpy as np
import pandas as pd
import sklearn
import torch


In [ ]:
device = "cpu"

Load preprocessed data


In [ ]:
def load_tweets(file_path):
    tweets = list()
    with open(file_path, 'r', encoding='utf-8') as preprocessed_tweets:
        for tweet in preprocessed_tweets :
            tweets.append(tweet.rstrip('\n'))
    return tweets

In [ ]:
    train_pos_tweets = load_tweets('/content/processed_pos_tweets_non_transformer.txt')
    train_neg_tweets = load_tweets('/content/processed_neg_tweets_non_transformer.txt')
    test_tweets = load_tweets('/content/processed_test_tweets_non_transformer.txt')
    print("Tweets loaded")

Tweets loaded


Convert to Dataframe

In [ ]:
    from sklearn.model_selection import train_test_split
    from sklearn.utils import shuffle
    seed =12222
    train_neg_labels = [0] * len(train_neg_tweets)
    train_pos_labels = [1] * len(train_pos_tweets)

    train_tweets = train_pos_tweets + train_neg_tweets
    train_labels = train_pos_labels + train_neg_labels
    #Shuffle
    train_tweets, train_labels = shuffle(train_tweets, train_labels, random_state=10)
    data = pd.DataFrame({'tweet': train_tweets, 'label': train_labels})

    X = list(data["tweet"])
    y = list(data["label"])
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.05,stratify=y,random_state=seed)
    X_train_set = pd.DataFrame({'tweet': X_train, 'label': y_train})
    X_val_set = pd.DataFrame({'tweet': X_val, 'label': y_val})

    X_test_set = pd.DataFrame({'tweet': test_tweets})


In [ ]:
from transformers import BertTokenizer, BertModel

# Load the Bertweet tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',max_len=128)
model = BertModel.from_pretrained('bert-base-uncased')
# model.to(device)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def embedding(tweet):
  return model.embeddings.word_embeddings.weight[tweet].mean(0) #to get BERT's output dimension, thus 678



def tokenize(tweet):
  return tokenizer.encode(tweet,max_length=128,truncation=True)


In [ ]:
from tqdm import tqdm

X_train = np.vstack([embedding(tokenize(z)).detach().cpu().numpy() for z in X_train])

In [ ]:
# import torch
import logging
import sklearn
from sklearn.ensemble import RandomForestClassifier
import xgboost
from xgboost import XGBClassifier
# model = RandomForestClassifier(n_estimators=200,max_depth=10,min_samples_split=30,verbose=True,n_jobs=-1,random_state=seed)
model_xgb =  XGBClassifier(n_estimators =850, tree_method = "gpu_hist", objective='binary:logistic', use_label_encoder=False,seed = seed)
model_xgb.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=850, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [ ]:
X_val = np.vstack([embedding(tokenize(z)).detach().cpu().numpy() for z in X_val])

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = model_xgb.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print("Validation accuracy:", accuracy)

Validation accuracy: 0.7771337832473204


In [ ]:
X_test = np.vstack([embedding(tokenize(z)).detach().cpu().numpy() for z in test_tweets])

In [ ]:
y_test = model_xgb.predict(X_test)
y_preds = [-1 if val == 0 else 1 for val in y_test]
df = pd.DataFrame(y_preds, columns=["Prediction"])
df.index.name = "Id"
df.index += 1
df.to_csv("/content/drive/MyDrive/test_data_bert_xgb.csv")